In [ ]:
import netCDF4 as nc
import numpy as np
import json
import math

In [ ]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
fn1 = 'sauce/Wind10.nc4' #NOAA GDAS dataset
ds1 = nc.Dataset(fn1)


# Processing NOAA GDAS dataset

Here we retrive neccessary data from NOAA GDAS dataset



In [ ]:
U_GRD_L103 = ds1["U_GRD_L103"][:] # U vector of the wind
V_GRD_L103 = ds1["V_GRD_L103"][:] # V vector of the wind
lat = ds1["lat"][:] # latitude
lon = ds1["lon"][:] # longtitude


NameError: ignored

# Processing CYGNSS dataset




In [ ]:
rad_fn = 'sauce/Sac1.nc4' #CYGNSS dataset
rad_ds = nc.Dataset(rad_fn)

In [ ]:
radds_lat = np.array(rad_ds["lat"][:])
radds_lon = np.array(rad_ds["lon"][:])

/var/folders/0g/k0tsczj93vq5fwg_pnt31zv40000gn/T/ipykernel_46745/898792714.py:2: UserWarning: WARNING: valid_range not used since it
cannot be safely cast to variable data type
  radds_lon = np.array(rad_ds["lon"][:])


### Calculating magnitude of wind vector and mss_modeled

In [ ]:
V = np.sqrt(ds1["U_GRD_L103"]['0'][6] * ds1["U_GRD_L103"]['0'][6]  + ds1["U_GRD_L103"]['0'][6] * ds1["U_GRD_L103"]['0'][6])
cond = V <= 3.49
''' Correlation between wind vector and mean square slope was taken from the paper (You can find it on github). '''
mss_mod_db = np.where((V <= 3.49), 0.0035 * (V + 0.62),  0.0035 * (6*np.log(V) - 3.39)) 


### Interpolating coordinates of CYGNSS dataset


In [ ]:
%%cython
from __main__ import radds_lat, radds_lon, lat, lon
import numpy
cimport numpy 




cdef float rad_var
cdef numpy.ndarray arr
cdef int arr_shape = arr.shape[0]


 
cdef radtomod(var, mod):
    for rad_var in numpy.nditer(var, op_flags = ['readwrite']):
        radtomod_ivar =  numpy.abs(mod - rad_var).argmin()
        rad_var[...] = mod[radtomod_ivar]
    return var

radds_lat = radtomod(radds_lat, lat)
radds_lon = radtomod(radds_lon, lon)



## Calculating microplastic density

In [ ]:
lst = np.array(lst)
mod_x = 0
fin_db = []

for rad_lat, rad_lon, mss_rad in zip(radds_lat, radds_lon, rad_ds["mean_square_slope"][:]):

        mod_i_lat = np.nonzero(lat == rad_lat)
        mod_i_lon = np.nonzero(lon == rad_lon)
        
        
        if not isinstance(mss_rad, np.float32):
                continue

        if V[mod_i_lat, mod_i_lon] < 3 or V[mod_i_lat, mod_i_lon] > 11:
                # print("Wrong V: ", V[mod_i_lat, mod_i_lon])
                continue
        mss_mod = mss_mod_db[mod_i_lat, mod_i_lon]
        mss_anom = (mss_rad / mss_mod) - 1
        
        dens = 2035*math.exp(-23.18*mss_anom)
        
        fin_db.append((rad_lat, rad_lon, dens))
        
        

## Inspector function
Helps to idendify problems with mss values on some coordiantes

In [ ]:
rad_lat = 3
rad_lon = -59.5

rad_lon += 180
rad_stack = np.stack((radds_lat, radds_lon), axis=1)

mod_i_lat = np.nonzero(lat == rad_lat)
mod_i_lon = np.nonzero(lon == rad_lon)

mod_i_rad = np.where((rad_stack == (rad_lat, rad_lon)).all(axis = 1))

if not isinstance(mss_rad, np.float32):
        print("Variables are not float")

if V[mod_i_lat, mod_i_lon] < 3 or V[mod_i_lat, mod_i_lon] > 11:
        print("Wrong V: ", V[mod_i_lat, mod_i_lon])
        
mss_rad = rad_ds["mean_square_slope"][mod_i_rad]
mss_mod = mss_mod_db[mod_i_lat, mod_i_lon]
mss_anom = (mss_rad / mss_mod) - 1


dens = 2035*np.exp(-23.18*mss_anom)

print("mod_i_rad: ", mod_i_rad)
print("V: ", V[mod_i_lat, mod_i_lon])
print('mss_rad: ', mss_rad, ' mss_mod: ', mss_mod, ' mss_anom: ', mss_anom)
print("Desnisty: ", dens)

print(dens)

## Converting data to json 

In [ ]:
json_db = []
for lat, lon, dens in fin_db:
  stg = {
        "type": "Feature",
        "properties": {
            "conScale": float(dens)
        },
        "geometry": {
          "type": "Point",
          "coordinates": [
            float(lat),
            float(lon - 180)
            
          ]
        }
    }
  json_db.append(stg)
json_str = {
  "type": "FeatureCollection",
  "features" : json_db 
}
with open("destination/data_base_large.json", "w") as write_file:
    json.dump(json_db, write_file, indent=4)

